### Build a simple LLM application with LCEL 

In this quickstart we'll show you how to build a simple LLM application with langchain.This application will translate text from english into another language.This is a relatively simple llm application - it's just a sigle llm call plus some prompting.still, this is a great way to get started with langchain -a lot of features can be built with just some prompting and an llm call! 

After seeing this notebook, you'll have a high level overview of:
- using language model 

- using PromptTemplates and outputParsers

- using <b>LangChain Expression Language</b> (LCEL) to chain components together

- debugging and tracing you application using langSmith 

- deploying your application with langServe

In [11]:
import os 
from rich import print
from dotenv import load_dotenv 
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [8]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

llm=ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10d5969e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10d744850>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [21]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from english to french."),
    HumanMessage(content="hello how are you?")

]
print(messages)

[
    SystemMessage(
        content='Translate the following from english to french.',
        additional_kwargs={},
        response_metadata={}
    ),
    HumanMessage(content='hello how are you?', additional_kwargs={}, response_metadata={})
]

In [22]:
result=llm.invoke(messages)

In [23]:
print(result.content)

Bonjour, comment allez-vous ? 


This is a formal way to say hello and ask how someone is doing in French. 

Here are some other options, depending on the context:

* **Informal:** Salut, ça va ?
* **Very informal:** Coucou, comment vas-tu ? (used with someone you know well)

In [24]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [26]:
print(parser.invoke(result))

Bonjour, comment allez-vous ? 


This is a formal way to say hello and ask how someone is doing in French. 

Here are some other options, depending on the context:

* **Informal:** Salut, ça va ?
* **Very informal:** Coucou, comment vas-tu ? (used with someone you know well)

#### Using LCEL- chain the components

In [27]:
chain=llm|parser

result=chain.invoke(messages)

In [28]:
print(result)

The most common way to say "Hello, how are you?" in French is:

**Bonjour, comment allez-vous ?** 

Here's a breakdown:

* **Bonjour:** Hello
* **comment:** how
* **allez-vous:** are you (formal, polite)


Let me know if you'd like other ways to say it!

#### Prompt Templates

In [29]:
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language} : "

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language} : '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [31]:
print(prompt.invoke({"language":"telugu","text":"hello how are you ? "}))

ChatPromptValue(
    messages=[
        SystemMessage(
            content='Translate the following into telugu : ',
            additional_kwargs={},
            response_metadata={}
        ),
        HumanMessage(content='hello how are you ? ', additional_kwargs={}, response_metadata={})
    ]
)

In [32]:
final_chain=prompt|llm|parser

In [34]:
print(final_chain.invoke({"language":"telugu","text":"hello how are you ?"}))

Here's how you say "Hello, how are you?" in Telugu:

**Formal:**

* **నమస్కారం, మీరు ఎలా ఉన్నారు?** (NamaskarAM, meeru ela unnaru?) 

**Informal:**

* **హలో, ఎలా ఉన్నావు?** (Halo, ela unnaavu?)

**Note:** 

*  "మీరు" (meeru) is the formal "you," used with respect.
* "మీ" (mee) is the more informal "you."
*  "ఎలా ఉన్నారు" (ela unnaru) is the formal way to ask "how are you?"
* "ఎలా ఉన్నావు" (ela unnaavu) is the informal way to ask "how are you?"


Let me know if you'd like to know more Telugu phrases!